In [2]:
import sys
sys.path

['D:\\cs_courses\\deep-learning\\my_exercises',
 '',
 'C:\\Users\\13123\\.conda\\envs\\deep\\lib\\site-packages',
 'C:\\Users\\13123\\.conda\\envs\\deep',
 'C:\\Users\\13123\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip',
 'C:\\Users\\13123\\AppData\\Local\\Programs\\Python\\Python312\\DLLs',
 'C:\\Users\\13123\\AppData\\Local\\Programs\\Python\\Python312\\Lib',
 'C:\\Users\\13123\\AppData\\Local\\Programs\\Python\\Python312',
 'C:\\Users\\13123\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages',
 'C:\\Users\\13123\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32',
 'C:\\Users\\13123\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\13123\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\Pythonwin']

In [3]:
import torch
import random

ImportError: Failed to load PyTorch C extensions:
    It appears that PyTorch has loaded the `torch/_C` folder
    of the PyTorch repository rather than the C extensions which
    are expected in the `torch._C` namespace. This can occur when
    using the `install` workflow. e.g.
        $ python setup.py install && python -c "import torch"

    This error can generally be solved using the `develop` workflow
        $ python setup.py develop && python -c "import torch"  # This should succeed
    or by running Python from a different directory.